<a href="https://colab.research.google.com/github/ogug8/Test_projects/blob/main/Stockprice_prediction.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [7]:
import pandas as pd
import yfinance as yf
import datetime
from datetime import date, timedelta
from plotly.offline import plot
today = date.today()

d1= today.strftime("%Y-%m-%d")
end_date =d1
d2 =date.today() - timedelta(days=768)
d2 =d2.strftime("%Y-%m-%d")
start_date = d2

data =yf.download("AAPL",start=start_date, end=end_date,progress=False)
data["Date"]=data.index
data = data[["Date","Open","High","Low","Close","Adj Close", "Volume"]]
data.reset_index(drop=True, inplace=True)

In [8]:
data.head()

,Date,Open,High,Low,Close,Adj Close,Volume
0,2022-01-31,170.160004,175.000000,169.509995,174.779999,172.585312,115541600
1,2022-02-01,174.009995,174.839996,172.309998,174.610001,172.417511,86213900
2,2022-02-02,174.750000,175.880005,173.330002,175.839996,173.632050,84914300
3,2022-02-03,174.479996,176.240005,172.119995,172.899994,170.728943,89418100
4,2022-02-04,171.679993,174.100006,170.679993,172.389999,170.442230,82465400


In [13]:
import plotly.graph_objects as go
figure = go.Figure(data=[go.Candlestick(x=data["Date"],
                                        open=data["Open"],
                                        high=data["High"],
                                        low=data["Low"],
                                        close=["Close"])])
figure.update_layout(title ="Apple Stock Price Analysis", xaxis_rangeslider_visible=False)
figure.show()

In [14]:
correlation =data.corr()
print(correlation["Close"].sort_values(ascending=False))

Close        1.000000
Adj Close    0.999677
Low          0.996954
High         0.996596
Open         0.991823
Volume      -0.469082
Name: Close, dtype: float64


<ipython-input-14-f8094e92b677>:1: FutureWarning:

The default value of numeric_only in DataFrame.corr is deprecated. In a future version, it will default to False. Select only valid columns or specify the value of numeric_only to silence this warning.



In [15]:
x= data[["Open","High","Low","Volume"]]
y= data["Close"]
x=x.to_numpy()
y = y.to_numpy()
y=y.reshape(-1,1)

from sklearn.model_selection import train_test_split
xtrain, xtest, ytrain, ytest = train_test_split(x,y,test_size=0.2,random_state=42)

In [18]:
from keras.models import Sequential
from keras.layers import Dense, LSTM
model=Sequential()
model.add(LSTM(128, return_sequences=True,input_shape= (xtrain.shape[1],1)))
model.add(LSTM(64,return_sequences=False))
model.add(Dense(25))
model.add(Dense(1))
model.summary()

Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lstm_2 (LSTM)               (None, 4, 128)            66560     
                                                                 
 lstm_3 (LSTM)               (None, 64)                49408     
                                                                 
 dense (Dense)               (None, 25)                1625      
                                                                 
 dense_1 (Dense)             (None, 1)                 26        
                                                                 
Total params: 117619 (459.45 KB)
Trainable params: 117619 (459.45 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [20]:
model.compile(optimizer='adam',loss='mean_squared_error')
model.fit(xtrain,ytrain,batch_size=1,epochs=30)

Epoch 1/30
422/422 [==============================] - 7s 8ms/step - loss: 5121.3823
Epoch 2/30
422/422 [==============================] - 4s 10ms/step - loss: 340.4350
Epoch 3/30
422/422 [==============================] - 4s 10ms/step - loss: 282.6818
Epoch 4/30
422/422 [==============================] - 3s 8ms/step - loss: 16.0631
Epoch 5/30
422/422 [==============================] - 5s 11ms/step - loss: 8.2173
Epoch 6/30
422/422 [==============================] - 4s 10ms/step - loss: 9.9816
Epoch 7/30
422/422 [==============================] - 3s 8ms/step - loss: 6.8818
Epoch 8/30
422/422 [==============================] - 4s 10ms/step - loss: 6.6634
Epoch 9/30
422/422 [==============================] - 3s 8ms/step - loss: 7.9783
Epoch 10/30
422/422 [==============================] - 3s 8ms/step - loss: 6.5185
Epoch 11/30
422/422 [==============================] - 3s 8ms/step - loss: 4.8690
Epoch 12/30
422/422 [==============================] - 4s 11ms/step - loss: 7.2163
Epoch 13/30

In [21]:
import numpy as np
#features = [Open,High,Low,adj Close,Volume]
features = np.array([[177.089996,180.419998,177.070007,74919600]])
model.predict(features)

1/1 [==============================] - 1s 1s/step


array([[178.90292]], dtype=float32)